In [5]:
import argparse
import os
import logging as log
import torch
import matplotlib
from dataloader.asimow_dataloader import DataSplitId, ASIMoWDataModule
from dataloader.latentspace_dataloader import LatentPredDataModule
from dataloader.utils import get_val_test_ids
from model.vq_vae import VectorQuantizedVAE
from model.vq_vae_patch_embedd import VQVAEPatch
from lightning.pytorch.loggers.wandb import WandbLogger
from lightning.pytorch.loggers.csv_logs import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning import Trainer
from model.mlp import MLP
from model.gru import GRU
from model.classification_model import ClassificationLightningModule
import numpy as np

In [15]:
dataset_dict = get_val_test_ids()
val_ids = dataset_dict["val_ids"]
test_ids = dataset_dict["test_ids"]

# Convert IDs to DataSplitId format and initialize data module
val_ids = [DataSplitId(experiment=e, welding_run=w) for e, w in val_ids]
test_ids = [DataSplitId(experiment=e, welding_run=w) for e, w in test_ids]
data_module = ASIMoWDataModule(task="reconstruction", batch_size=512, n_cycles=1, val_data_ids=val_ids, test_data_ids=test_ids)
input_dim = 2  # Ensure input dimension is set correctly for 'asimow'
data_module.setup(stage="fit")

# Get DataLoader
train_loader = data_module.test_dataloader()

In [16]:
for i, batch in enumerate(train_loader):
    # Here, we're assuming each batch is a single tensor
    data = batch

    print(f"Batch {i}:")
    print(f"Shape: {data.shape}")

    # Print the first few samples for inspection
    print("First few samples:")
    print(data[:5])

    if i >= 1:  # Limit to inspecting a couple of batches
        break


In [34]:
import pickle
import pandas as pd

# Path to your .pickle file
pickle_file_path = "data/quality_prediction_data/asimow/dataset.pickle"

# Load the dataset from the pickle file
with open(pickle_file_path, 'rb') as file:
    dataset = pickle.load(file)

ex_1_and_2 = dataset[(dataset.experiment!=3)]
ex_3 = dataset[(dataset.experiment==3)]

# Now, 'dataset' is the object that was stored in the .pickle file.
# You can use it as per your requirements.
